# Use reachy-sdk to control Reachy and it's mobile base at the same time

So far in the other notebooks we presented how to control Reachy's mobile base alone, but it is of course possible to control it along with your Reachy!

To do that, you will need to use [reachy-sdk](https://github.com/pollen-robotics/reachy-sdk), our Python SDK to control Reachy.

In [ ]:
from reachy_sdk import ReachySDK

In the ReachySDK constructor, specify that your Reachy is equipped with a mobile base and if Reachy SDK server and the mobile base SDK server are running, you're good to go.

In [ ]:
reachy = ReachySDK(host='192.168.86.35', with_mobile_base=True) # Replace the IP with your Reachy's IP

You have now access to the mobile base with *reachy.mobile_base*!

In [ ]:
reachy.mobile_base

As in the other notebooks, you can send goto instructions to the mobile base, access its odometry, battery voltage,...

In [ ]:
reachy.mobile_base.reset_odometry()

In [ ]:
reachy.mobile_base.goto(x=0, y=0, theta=-45)

In [ ]:
reachy.mobile_base.battery_voltage

And as usual you can control your Reachy!

In [ ]:
reachy

In [ ]:
reachy.turn_on('head')

reachy.head.look_at(0.5, 0, -0.5, 1.0)

reachy.head.look_at(0.5, 0, 0, 1.0)

## [advanced] Move Reachy and the mobile base at the same time

What we will show you here is an advanced example of movements you can do by combining goto instructions on Reachy's arm and sending goto to the mobile base.

We will show you how you can do a movement like this:

In [ ]:
from IPython.display import HTML

HTML("""
    <video width="25%" controls>
        <source src="data/goto_reachy_and_mobile_base.mp4" type="video/mp4">
    </video>
""")

To do that, we will have to use the asynchronous version of the goto function of Reachy and the goto of the mobile base.

If you're not familiar with asynchronous programming, you can check the [Python documentation](https://docs.python.org/3/library/asyncio.html) of *asyncio*, the Python library to do asynchronous programming.

In [ ]:
import asyncio

from reachy_sdk.trajectory import goto_async
from reachy_sdk.trajectory.interpolation import InterpolationMode

import numpy as np

The sequence showed in the video is composed of two parts:
* goto instruction for the mobile base in its odom frame,
* greeting movement from Reachy's right arm which is itself combining mutiple gotos

### goto for the mobile base

This one is the easy part. We will just define a target point (x, y, theta) in the odom frame of the mobile base. To make the movement that you saw on the video above, we took x=-0.3m, y=-1.0m and theta=35°.

Remember that x and y are in meters and that theta is in degree.

We can define the aynschronous goto for the mobile base like this:

In [ ]:
goto_async_mobile_base = reachy.mobile_base.goto_async(x=-0.3, y=-1.0, theta=35.0, timeout=2.0)

**Note**: just executing this command will not make the mobile base move as it is an asynchronous command. If you want to make the mobile base execute the goto we just defined, you will have to use the *await* keyword.

In [ ]:
await goto_async_mobile_base

### greetings movement for Reachy's right arm

This movement is a bit more complicated as it is composed of:
* one movement to raise the arm
* agitation of Reachy's wrist
* one movement to put the arm along Reachy's torso

#### Raising the arm

To have the first movement, we actually put Reachy's right arm in compliant mode in the final position we wanted it to be and saved the present position of each joint's arm. This saved positions will be given to the goto_async of ReachySDK to send the command.

```python
greet_joint_pos = [5, -15, -65, -131, 83, 26, -16]

# Build a dictionnary with each Reachy's joint we want to move and its goal position
goal_goto_greet = {joint: gp for (joint, gp) in zip(list(reachy.r_arm.joints.values()), greet_joint_pos)}

goto_greet = goto_async(
        goal_positions=goal_goto_greet,
        duration=1.2,
        interpolation_mode=InterpolationMode.MINIMUM_JERK,
    )
```

#### Agitating Reachy's wrist

To do this, we just define two goal positions for the wrist_roll which is the only joint involved in the movement. We got these values by doing the same thing as before: we placed the joint in compliant mode in the position we wanted it to be. Then we will just switch between the positions to have the agitation of the wrist.

```python
goal_1_wrist = {reachy.r_arm.r_wrist_roll: 23.0}
goal_2_wrist = {reachy.r_arm.r_wrist_roll: -17.0}

goto_wrist_1 = goto_async(
        goal_positions=goal_1_wrist,
        duration=0.5,
        interpolation_mode=InterpolationMode.MINIMUM_JERK,
    )
goto_wrist_2 = goto_async(
        goal_positions=goal_2_wrist,
        duration=0.5,
        interpolation_mode=InterpolationMode.MINIMUM_JERK,
    )
```

#### Putting Reachy's arm along its torso

To do that, we just an async goto with 0.0 as goal position for each joint.

```python
goal_goto_rest = {joint: 0 for joint in list(reachy.r_arm.joints.values())}

goto_rest = goto_async(
        goal_positions=goal_goto_rest,
        duration=1.5,
        interpolation_mode=InterpolationMode.MINIMUM_JERK,
    )

```

#### Wrapping everything in one function

Now that each part of the greetings movement is set, we can define a function that will perform the greetings.

In [ ]:
# Raising the arm
greet_joint_pos = [5, -15, -65, -131, 83, 26, -16]
goal_goto_greet = {joint: gp for (joint, gp) in zip(list(reachy.r_arm.joints.values()), greet_joint_pos)}


# Agitating the wrist
goal_1_wrist = {reachy.r_arm.r_wrist_roll: 23.0}
goal_2_wrist = {reachy.r_arm.r_wrist_roll: -17.0}


# Putting Reachy's arm along its torso
goal_goto_rest = {joint: 0 for joint in list(reachy.r_arm.joints.values())}


async def greet(reachy):
    reachy.turn_on('r_arm')

    # Raising the arm
    await goto_async(
        goal_positions=goal_goto_greet,
        duration=1.5,
        interpolation_mode=InterpolationMode.MINIMUM_JERK,
    )

    # Agitating the wrist
    await goto_async(
            goal_positions=goal_1_wrist,
            duration=0.5,
            interpolation_mode=InterpolationMode.MINIMUM_JERK,
        )
    await goto_async(
            goal_positions=goal_2_wrist,
            duration=0.5,
            interpolation_mode=InterpolationMode.MINIMUM_JERK,
        )
    await goto_async(
            goal_positions=goal_1_wrist,
            duration=0.5,
            interpolation_mode=InterpolationMode.MINIMUM_JERK,
        )
    await goto_async(
            goal_positions=goal_2_wrist,
            duration=0.5,
            interpolation_mode=InterpolationMode.MINIMUM_JERK,
        )

    # Putting Reachy's arm along its torso
    await goto_async(
        goal_positions=goal_goto_rest,
        duration=1.5,
        interpolation_mode=InterpolationMode.MINIMUM_JERK,
    )
    reachy.turn_off_smoothly('r_arm')

In [ ]:
await greet(reachy)

### Moving Reachy's arm and its mobile base at the same time

Now it's time to reproduce the example showed in the video!

To do that, you just have to use *gather* from asyncio.

In [ ]:
# First make sure that the mobile base is at the center of its odom frame
reachy.mobile_base.goto(x=0.0, y=0.0, theta=0.0)

In [ ]:
await asyncio.gather(
    greet(reachy),
    goto_async_mobile_base,
)

That's it! You know how to make the mobile base and the rest of Reachy move at the same time!